In [1]:
import pandas as pd
import os
import numpy as np
import yaml
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from category_encoders import CatBoostEncoder
import lazypredict
from lazypredict.Supervised import LazyRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from yellowbrick.regressor import ResidualsPlot
from sklearn.pipeline import Pipeline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
#random seed
random.seed(42)
config_path = os.path.join(os.getcwd(), 'config\config.yaml')

In [2]:
with open(config_path) as f:
    config = yaml.safe_load(f)

In [3]:
df = pd.read_csv("data_tidy.csv")
df.shape

(12513, 28)

In [4]:
#lowercase if object
df = df.apply(lambda x: x.astype(str).str.lower() if x.dtype == "object" else x)

Drop null những MISC_Price_Euro có giá trị null

In [5]:
#drop na in "MISC_Price_Euro"
df = df.dropna(subset=["MISC_Price_Euro"])
df.shape

(8438, 28)

In [6]:
X = df.drop(["MISC_Price_Euro"], axis=1)
y = df["MISC_Price_Euro"]


In [7]:
# # X['resolution'] = DISPLAY_Resolution_Width*DISPLAY_Resolution_Height
# X['resolution'] = X['DISPLAY_Resolution_Width']*X['DISPLAY_Resolution_Height']
# #drop
# X = X.drop(["DISPLAY_Resolution_Width", "DISPLAY_Resolution_Height"], axis=1)

In [8]:
X = X.drop(["NETWORK_GPRS", "NETWORK_EDGE"], axis=1)

#### Catboost Enc features

In [9]:
cat_enc_cols = config["cat_enc_cols"]
cat_enc_cols

['Brand',
 'NETWORK_Technology',
 'NETWORK_2G_bands',
 'NETWORK_3G_bands',
 'NETWORK_4G_bands',
 'NETWORK_5G_bands',
 'BATTERY_Type',
 'MAIN_CAM_1_Module',
 'MAIN_CAM_1_Video',
 'DISPLAY_Type',
 'PLATFORM_OS',
 'MEMORY_Card_slot']

In [10]:
cat_enc_pipe = Pipeline([
    ('enc', CatBoostEncoder())
])
for col in cat_enc_cols:
    X[col] = cat_enc_pipe.fit_transform(X[col], y)

#### Enc for labels 

In [11]:
# df.FEATURES_Sensors = df.FEATURES_Sensors.str.lower()
# def find_unique_values(series):
#     unique_values = set()
#     for value in series:
#         if isinstance(value, str):  # Only process string values
#             sensors = value.split(', ')
#             for sensor in sensors:
#                 unique_values.add(sensor)
#     return list(unique_values)

# sensors = find_unique_values(df.FEATURES_Sensors)
# sensors = sorted(sensors)
# len(sensors)

In [12]:
# # Split the 'FEATURES_Sensors' column into separate sensors
# df_sensors = df['FEATURES_Sensors'].str.split(', ', expand=True).stack()
# #lowercase

# # Create a new DataFrame with the individual sensors and the corresponding 'MISC_Price_Euro' values
# df_individual_sensors = df.loc[df_sensors.index.get_level_values(0), ['MISC_Price_Euro']].copy()
# df_individual_sensors['Sensor'] = df_sensors.values

# # Calculate the mean 'MISC_Price_Euro' for each sensor
# sensor_means = df_individual_sensors.groupby('Sensor')['MISC_Price_Euro'].mean()

# # Now sensor_means is a Series where the index is the sensor name and the value is the mean 'MISC_Price_Euro'
# sensor_means

In [13]:
def calculate_sensor_means(df, column):
    # Split the column into separate sensors
    df[column] = df[column].str.lower()
    df_sensors = df[column].str.split(', ', expand=True).stack()
    # lowercase all values
    df_sensors = df_sensors.str.lower()
    # Create a new DataFrame with the individual sensors and the corresponding 'MISC_Price_Euro' values
    df_individual_sensors = df.loc[df_sensors.index.get_level_values(0), ['MISC_Price_Euro']].copy()
    df_individual_sensors['Sensor'] = df_sensors.values

    # Calculate the mean 'MISC_Price_Euro' for each sensor
    sensor_means = df_individual_sensors.groupby('Sensor')['MISC_Price_Euro'].mean()

    # Replace the values in the column with the mean 'MISC_Price_Euro' for each sensor
    df[column] = df[column].apply(lambda x: np.mean([sensor_means.get(i, 0) for i in str(x).split(', ')]))
    #fill
    df[column].replace(0, df[column].mean(), inplace=True)
    #add to X
    X[column] = df[column]
    return df

# Usage:
df = calculate_sensor_means(df, 'NETWORK_Speed')
df = calculate_sensor_means(df, 'FEATURES_Sensors')

In [14]:
X.isnull().sum()

Brand                           0
NETWORK_Technology              0
NETWORK_2G_bands                0
NETWORK_3G_bands                0
NETWORK_4G_bands                0
NETWORK_5G_bands                0
NETWORK_Speed                   0
LAUNCH_Announced               11
BODY_Weight                   544
BODY_Length                   197
BODY_Width                    196
BODY_Thickness                167
FEATURES_Sensors                0
DISPLAY_Type                    0
DISPLAY_Size                   90
DISPLAY_Resolution_Width       13
DISPLAY_Resolution_Height      12
PLATFORM_OS                     0
MEMORY_Card_slot                0
MEMORY_Internal_rom           561
MEMORY_Internal_ram          1783
BATTERY_Type                    0
BATTERY_Capacity              261
MAIN_CAM_1_Module               0
MAIN_CAM_1_Video                0
dtype: int64

#### FILLIN

In [15]:
from sklearn.impute import KNNImputer

In [16]:
# Specify the columns to impute
columns_to_impute = ['BODY_Weight', 'BODY_Length', 'BODY_Width', 'BODY_Thickness', 
                     'MEMORY_Internal_rom', 'MEMORY_Internal_ram', 'LAUNCH_Announced', 
                     'DISPLAY_Resolution_Width', 'DISPLAY_Resolution_Height', 'DISPLAY_Size', 'BATTERY_Capacity']

# Create the imputer
imputer = KNNImputer(n_neighbors=3)

# Apply the imputer to the specified columns
X_filled = imputer.fit_transform(X[columns_to_impute])

# Update the original DataFrame with the imputed values
X[columns_to_impute] = X_filled

#### drop outlier

In [17]:
#corr toward y 
corr = X_scaled.corrwith(y_scaled).sort_values(ascending=False)
corr

NameError: name 'X_scaled' is not defined

In [18]:
#check skewness
X.skew()
#preprocess for skew

Brand                       15.22
NETWORK_Technology           1.74
NETWORK_2G_bands             0.52
NETWORK_3G_bands             2.37
NETWORK_4G_bands             6.66
NETWORK_5G_bands             6.72
NETWORK_Speed                1.24
LAUNCH_Announced            -0.12
BODY_Weight                  3.78
BODY_Length                  0.90
BODY_Width                   2.64
BODY_Thickness               1.44
FEATURES_Sensors            -0.41
DISPLAY_Type                 1.55
DISPLAY_Size                 0.47
DISPLAY_Resolution_Width     0.89
DISPLAY_Resolution_Height    0.27
PLATFORM_OS                 26.30
MEMORY_Card_slot             1.82
MEMORY_Internal_rom          2.37
MEMORY_Internal_ram          5.89
BATTERY_Type                 0.30
BATTERY_Capacity             1.44
MAIN_CAM_1_Module           -4.06
MAIN_CAM_1_Video             1.77
dtype: float64

In [19]:
#check skewness if high, log transform
for col in X.columns:
    if X[col].skew() > 1:
        X[col] = np.log1p(X[col])

In [20]:
X.head()

,Brand,NETWORK_Technology,NETWORK_2G_bands,NETWORK_3G_bands,NETWORK_4G_bands,NETWORK_5G_bands,NETWORK_Speed,LAUNCH_Announced,BODY_Weight,BODY_Length,BODY_Width,BODY_Thickness,FEATURES_Sensors,DISPLAY_Type,DISPLAY_Size,DISPLAY_Resolution_Width,DISPLAY_Resolution_Height,PLATFORM_OS,MEMORY_Card_slot,MEMORY_Internal_rom,MEMORY_Internal_ram,BATTERY_Type,BATTERY_Capacity,MAIN_CAM_1_Module,MAIN_CAM_1_Video
0,5.43,5.43,227.52,5.43,5.43,5.43,4.99,2018.00,6.30,238.30,5.15,2.39,254.27,5.43,9.70,1536.00,2048.00,5.43,5.43,17.33,15.25,227.52,8.41,1,5.43
1,5.64,5.43,227.52,5.43,5.43,5.64,5.28,2016.00,5.56,191.70,4.62,2.34,256.87,5.64,7.00,720.00,1280.00,5.43,5.64,17.33,14.56,227.52,8.13,1,5.43
2,5.50,5.30,227.52,5.43,5.43,5.50,5.50,2016.00,5.14,153.80,4.34,2.25,278.52,5.50,5.50,1080.00,1920.00,5.30,5.50,17.33,14.96,281.63,8.31,1,5.30
3,5.51,5.38,240.00,5.48,5.48,5.51,5.48,2016.00,4.84,145.50,4.30,2.25,256.87,5.51,5.00,720.00,1280.00,5.38,5.51,15.94,13.86,198.87,7.60,1,5.43
4,5.41,5.64,281.63,5.64,5.64,5.41,4.99,2016.00,6.04,259.00,5.12,2.29,254.27,5.41,10.10,1920.00,1200.00,5.27,5.41,18.02,16.64,173.25,8.72,1,5.38


#### test w/ lazy_predict

In [21]:
#standardScaler
from sklearn.preprocessing import StandardScaler

In [22]:
scaler = StandardScaler()
X = scaler.fit_transform(X,y)
y_scaled = scaler.fit_transform(y.values.reshape(-1,1))
#log1p scale for y
# y = np.log1p(y)
y_scaled = y
X_scaled = pd.DataFrame(X, columns=config['variables'])
X_scaled.head()

,Brand,NETWORK_Technology,NETWORK_2G_bands,NETWORK_3G_bands,NETWORK_4G_bands,NETWORK_5G_bands,NETWORK_Speed,LAUNCH_Announced,BODY_Weight,BODY_Length,BODY_Width,BODY_Thickness,FEATURES_Sensors,DISPLAY_Type,DISPLAY_Size,PLATFORM_OS,MEMORY_Card_slot,MEMORY_Internal_rom,MEMORY_Internal_ram,BATTERY_Type,BATTERY_Capacity,MAIN_CAM_1_Module,MAIN_CAM_1_Video,DISPLAY_Resolution_Width,DISPLAY_Resolution_Height
0,-0.07,0.22,0.05,0.21,0.50,0.42,-1.24,0.55,2.66,2.51,3.08,-0.13,-0.02,0.20,2.20,2.03,0.94,0.41,0.22,0.47,0.61,-0.04,0.90,0.23,0.18
1,0.37,0.22,0.05,0.21,0.50,1.51,-0.39,0.14,1.10,1.28,1.26,-0.31,0.01,0.60,0.94,0.04,0.00,0.41,0.72,0.47,0.31,-0.04,0.51,0.23,0.18
2,0.08,-0.03,0.05,0.21,0.50,0.78,0.26,0.14,0.20,0.29,0.28,-0.66,0.29,0.34,0.25,0.92,0.78,0.14,0.39,0.47,0.48,0.91,0.77,0.23,-0.08
3,0.09,0.13,0.22,0.35,0.66,0.83,0.20,0.14,-0.42,0.07,0.13,-0.66,0.01,0.35,0.01,0.04,0.00,0.31,0.41,0.04,0.01,-0.54,-0.23,0.23,0.18
4,-0.12,0.64,0.81,0.76,1.17,0.28,-1.24,0.14,2.11,3.06,2.97,-0.50,-0.02,0.15,2.38,2.96,-0.09,0.09,0.16,0.68,1.21,-0.99,1.33,0.23,0.09


In [23]:
regressors = config['regressors']
#removed regressors are those not in regressors 
removed_regressors = [est[0] for est in all_estimators() if (est[0] not in regressors)]

regressor_list = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (est[0] not in removed_regressors))
]
def lazy_eval(X_train, y_train, X_test, y_test):
    reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None,predictions=False, regressors = regressor_list)
    models, predictions = reg.fit(X_train, X_test, y_train, y_test)
    return models, predictions
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=0)

In [148]:
models, predictions = lazy_eval(X_train, y_train, X_test, y_test)
models

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 9/9 [00:36<00:00,  4.10s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.81,0.81,0.32,6.57
RandomForestRegressor,0.81,0.81,0.32,20.24
SVR,0.79,0.79,0.34,2.71
GradientBoostingRegressor,0.78,0.78,0.35,5.72
KNeighborsRegressor,0.75,0.75,0.37,0.14
LinearRegression,0.70,0.70,0.41,0.03
SGDRegressor,0.70,0.70,0.41,0.04
AdaBoostRegressor,0.65,0.65,0.44,1.20
DecisionTreeRegressor,0.63,0.63,0.45,0.27


In [149]:
model = ExtraTreesRegressor(random_state=42)
model.fit(X_train, y_train)
# Make predictions and evalute
y_pred = model.predict(X_test)
print("R^2: {}".format(model.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
print("MSE: {}".format(mean_squared_error(y_test, y_pred)))

R^2: 0.8142422005586928
Root Mean Squared Error: 0.3212372773928405
MSE: 0.10319338838676476


#### tuning

In [24]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import numpy as np

#### ExtraTree 

In [156]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import numpy as np

# Define the pipeline
pipe = Pipeline([
    ('reg', ExtraTreesRegressor(random_state=42))
])

# Define the function to optimize
def evaluate(n_estimators, max_depth):
    max_depth = int(max_depth)
    if max_depth == 0:
        max_depth = None
    pipe.set_params(reg__n_estimators=int(n_estimators),
                    reg__max_depth=max_depth)
    return np.mean(cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2'))

# Define the bounds of the parameters
param_bounds = {
    'n_estimators': (50, 150),
    'max_depth': (0, 10),  # 0 will be interpreted as None
}

# Create the Bayesian Optimization object
optimizer = BayesianOptimization(
    f=evaluate,
    pbounds=param_bounds,
    random_state=1,
)

# Perform the optimization
optimizer.maximize(init_points=10, n_iter=20)

# Get the best parameters
best_params = optimizer.max['params']
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])

print(best_params)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.6461    | 4.17      | 122.0     |
| 2         | 0.8087    | 0.001144  | 80.23     |
| 3         | 0.3766    | 1.468     | 59.23     |
| 4         | 0.3806    | 1.863     | 84.56     |
| 5         | 0.5923    | 3.968     | 103.9     |
| 6         | 0.646     | 4.192     | 118.5     |
| 7         | 0.5239    | 2.045     | 137.8     |
| 8         | 0.8092    | 0.2739    | 117.0     |
| 9         | 0.6458    | 4.173     | 105.9     |
| 10        | 0.3773    | 1.404     | 69.81     |
| 11        | 0.8085    | 0.0       | 78.08     |
| 12        | 0.8093    | 0.0       | 113.2     |
| 13        | 0.6456    | 4.662     | 112.2     |
| 14        | 0.7833    | 10.0      | 150.0     |
| 15        | 0.6453    | 4.804     | 150.0     |
| 16        | 0.7834    | 10.0      | 145.2     |
| 17        | 0.7833    | 10.0      | 139.6     |
| 18        | 0.7738    | 9.971     | 133.4     |


In [157]:
best_params['max_depth'] = None
pipe = Pipeline([
    ('reg', ExtraTreesRegressor(**best_params)
)])
pipe.fit(X_train, y_train)
print('R2: {:.4f}'.format(pipe.score(X_test, y_test)))
print('MSE: {:.4f}'.format(mean_squared_error(y_test, pipe.predict(X_test))))
print('MAE: {:.4f}'.format(mean_absolute_error(y_test, pipe.predict(X_test))))

R2: 0.8129
MSE: 0.1039
MAE: 0.2372


In [1030]:
# pipe = Pipeline([
#     ('transform', PolynomialFeatures(1)),
#     ('reg', ExtraTreesRegressor(105))
# ])

# pipe.fit(X_train, y_train)

# r2_linear = r2_score(y_test, pipe.predict(X_test))
# mse_linear = mean_squared_error(y_test, pipe.predict(X_test))
# print(f"R^2: {r2_linear:.4f}")
# print(f"MSE: {mse_linear:.4f}")

R^2: 0.5814
MSE: 0.3597


# linear regression model

In [25]:
from sklearn.preprocessing import PolynomialFeatures
# Define the pipeline
pipe = Pipeline([
    ('transform', PolynomialFeatures()),
    ('reg', LinearRegression())
])

# Define the function to optimize
def evaluate(degree, fit_intercept, positive):
    pipe.set_params(transform__degree=int(degree), reg__fit_intercept=bool(fit_intercept),
                    reg__positive=bool(positive))
    return np.mean(cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2'))

# Define the bounds of the parameters
param_bounds = {
    'degree': (1, 3),
    'fit_intercept': (0, 1),  # 0 for False, 1 for True
    'positive': (0, 1)
    
}

# Create the Bayesian Optimization object
optimizer = BayesianOptimization(
    f=evaluate,
    pbounds=param_bounds,
    random_state=1,
)

# Perform the optimization
optimizer.maximize(init_points=10, n_iter=2)

# Get the best parameters
best_params = optimizer.max['params']
best_params['degree'] = int(best_params['degree'])
best_params['fit_intercept'] = bool(int(best_params['fit_intercept']))
print(best_params)

|   iter    |  target   |  degree   | fit_in... | positive  |
-------------------------------------------------------------
| 1         | 0.2459    | 1.834     | 0.7203    | 0.0001144 |
| 2         | 0.2459    | 1.605     | 0.1468    | 0.09234   |
| 3         | 0.2459    | 1.373     | 0.3456    | 0.3968    |
| 4         | 0.4776    | 2.078     | 0.4192    | 0.6852    |
| 5         | 0.2459    | 1.409     | 0.8781    | 0.02739   |
| 6         | 0.4776    | 2.341     | 0.4173    | 0.5587    |
| 7         | 0.2459    | 1.281     | 0.1981    | 0.8007    |
| 8         | 0.4776    | 2.937     | 0.3134    | 0.6923    |
| 9         | 0.4776    | 2.753     | 0.8946    | 0.08504   |
| 10        | 0.2459    | 1.078     | 0.1698    | 0.8781    |
| 11        | 0.4776    | 2.588     | 1.0       | 1.0       |
| 12        | 0.4776    | 2.361     | 0.1212    | 0.9838    |
{'degree': 2, 'fit_intercept': False, 'positive': 0.6852195003967595}


In [26]:
pipe = Pipeline([
    ('transform', PolynomialFeatures(degree=best_params['degree'])),
    ('reg', LinearRegression(fit_intercept=best_params['fit_intercept']))
])

# Fit the model
pipe.fit(X_train, y_train)

# Evaluate on the test set
r2_linear = r2_score(y_test, pipe.predict(X_test))
mse_linear = mean_squared_error(y_test, pipe.predict(X_test))
print(f"R^2: {r2_linear:.4f}")
print(f"MSE: {mse_linear:.4f}")
print(f"MAE: {mean_absolute_error(y_test, pipe.predict(X_test)):.4f}")

R^2: 0.3942
MSE: 64966.6265
MAE: 109.4830


#### KNNREGRESSOR

In [27]:
from sklearn.neighbors import KNeighborsRegressor

# Define the pipeline
pipe = Pipeline([
    # ('transform', PolynomialFeatures()),
    ('reg', KNeighborsRegressor())
])

# Define the function to optimize
def evaluate(n_neighbors, p):
    pipe.set_params(reg__n_neighbors=int(n_neighbors), reg__p=int(p))
    return np.mean(cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2'))

# Define the bounds of the parameters
param_bounds = {
    'n_neighbors': (1, 10),
    'p': (1, 3),
}

# Create the Bayesian Optimization object
optimizer = BayesianOptimization(
    f=evaluate,
    pbounds=param_bounds,
    random_state=1,
)

# Perform the optimization
optimizer.maximize(init_points=5, n_iter=25)

# Get the best parameters
best_params = optimizer.max['params']
best_params['n_neighbors'] = int(best_params['n_neighbors'])
best_params['p'] = int(best_params['p'])

|   iter    |  target   | n_neig... |     p     |
-------------------------------------------------
| 1         | 0.4763    | 4.753     | 2.441     |
| 2         | 0.3737    | 1.001     | 1.605     |
| 3         | 0.4283    | 2.321     | 1.185     |
| 4         | 0.4283    | 2.676     | 1.691     |
| 5         | 0.4763    | 4.571     | 2.078     |
| 6         | 0.3779    | 6.816     | 1.0       |
| 7         | 0.4684    | 10.0      | 3.0       |
| 8         | 0.3427    | 10.0      | 1.221     |
| 9         | 0.4752    | 3.796     | 3.0       |
| 10        | 0.4805    | 8.381     | 3.0       |
| 11        | 0.4794    | 6.699     | 3.0       |
| 12        | 0.3984    | 4.004     | 1.0       |
| 13        | 0.4847    | 7.537     | 3.0       |
| 14        | 0.396     | 2.479     | 3.0       |
| 15        | 0.4737    | 5.645     | 2.997     |
| 16        | 0.4733    | 9.202     | 3.0       |
| 17        | 0.5228    | 3.989     | 2.369     |
| 18        | 0.5228    | 3.631     | 2.203     |


In [28]:
pipe = Pipeline([
    # ('transform', PolynomialFeatures(degree=best_params['degree'])),
    ('reg', KNeighborsRegressor(n_neighbors=best_params['n_neighbors'], p=best_params['p']))
])

# Fit the model
pipe.fit(X_train, y_train)

# Evaluate on the test set
r2_knn = r2_score(y_test, pipe.predict(X_test))
mse_knn = mean_squared_error(y_test, pipe.predict(X_test))
print(f"R^2: {r2_knn:.4f}")
print(f"MSE: {mse_knn:.4f}")
print(f"MAE: {mean_absolute_error(y_test, pipe.predict(X_test)):.4f}")
    

R^2: 0.4104
MSE: 63230.8899
MAE: 70.0334
